# CCAMMLR List Production

The following downloads the CCAMLR list from BigQuery, inspects the geartypes, and then maps the geartypes to the labels we decided on for the neural net input. See the readme.md file in this folder for more on this mapping

In [1]:
import pandas as pd
import csv
import pandas.io.gbq as bq

In [2]:
# first find the geartypes

q = '''SELECT  geartype FROM [world-fishing-827:Registry_matching_sources.CCAMLR_July_2015_with_mmsi] group by geartype
'''

cc_gear =pd.io.gbq.read_gbq(q, project_id='world-fishing-827', verbose = False )

In [4]:
cc_gear

,geartype
0,Longlines (not specified)
1,Longlines (not specified); Other trawls (not s...
2,Longlines (not specified); Pots
3,Other trawls (not specified)
4,Midwater trawls (not specified)


In [8]:
cc_gear.loc[1]['geartype']

u'Longlines (not specified); Other trawls (not specified)'

In [9]:
gear_map = {'Longlines (not specified)':'Drifting_longlines',
           'Longlines (not specified); Pots':"Drifting_longlines|Pots_and_traps",
           'Longlines (not specified); Other trawls (not specified)':'Drifting_longlines|Trawlers',
           'Other trawls (not specified)':"Trawlers",
          'Midwater trawls (not specified)':'Trawlers' }

In [10]:

q = '''SELECT
  mmsi,
  geartype,
  tonnage,
  length
FROM
  [world-fishing-827:Registry_matching_sources.CCAMLR_July_2015_with_mmsi]
'''

ccamlr =pd.io.gbq.read_gbq(q, project_id='world-fishing-827', verbose = False )

In [12]:
ccamlr['label']=ccamlr['geartype'].map(gear_map)

In [16]:
ccamlr = ccamlr.set_index('mmsi')

In [17]:
ccamlr.head()

,geartype,tonnage,length,label
mmsi,,,,
503595000,Longlines (not specified),1136,62.8,Drifting_longlines
503552000,Longlines (not specified); Other trawls (not s...,1906,68.1,Drifting_longlines|Trawlers
503556000,Longlines (not specified); Pots,1045,59.8,Drifting_longlines|Pots_and_traps
503000087,Longlines (not specified),1155,54.58,Drifting_longlines
508770000,Longlines (not specified),1362,49.9,Drifting_longlines


In [19]:
ccamlr[['label','tonnage','length']].to_csv('../classification-list-sourcesccamlr.csv')